# Assignment 1
## Problem 6 - Normalizing Flows

In [ ]:

%reload_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import pyro.distributions as dist
import torch
from matplotlib import cm
from pyro.distributions.transforms import spline_coupling, spline_autoregressive, permute
from pyro.nn import PyroModuleList

from src.utils.io import load_image, img_to_bw
from src.utils.plotting import init_plot_style, show_grayscale_img

init_plot_style()
data_dir = '../../data/img/'

If necessary, we first have to convert our grayscale icons to a binary image.

In [ ]:
img = img_to_bw(data_dir + 'up.png', data_dir + 'up_bw.png')
show_grayscale_img(img)

Now we can load the binary image and generate samples from it as described in the assignment sheet.

In [ ]:
# load image
img = load_image(data_dir + 'up_bw.png')
height, width = img.shape
print(f'Our image format is {width} x {height}.')

# generate training samples
n_samples = 1000  # number of samples to generate
data = torch.zeros((n_samples, 2))
i = 0
while i < n_samples:
    row = torch.randint(height, (1,))
    col = torch.randint(width, (1,))
    if img[row, col] == 0:
        data[i, 0] = col
        data[i, 1] = row
        i += 1

# plot image with generated samples
show_grayscale_img(img)
plt.plot(data[:, 0], data[:, 1], 'x')
plt.xlim([0, width])
plt.ylim([0, height])


def normalise(x: torch.Tensor):
    return 2. * x / torch.tensor([width, height]) - 1.


def unnormalise(x: torch.Tensor):
    return 0.5 * (x + 1.) * torch.tensor([width, height])


# for training a NN its important to normalise your input data!
data = normalise(data)


Build and train your flow here.

In [ ]:


# ToDo: build the model

# you may want to utilise spline_coupling + permute or spline_autoregressive
# as a final transformation you may consider using TanhTransform
transforms = []






# setup training
modules = PyroModuleList([t for t in transforms if isinstance(t, torch.nn.Module)])
optimizer = torch.optim.Adam(modules.parameters(), lr=1e-2, weight_decay=1e-4)

# train the flow
steps = 2000
losses = []
for step in range(steps + 1):
    optimizer.zero_grad()
    loss = -flow_dist.log_prob(data).mean()
    loss.backward()
    optimizer.step()
    flow_dist.clear_cache()

    losses.append(loss.item())
    if step % 100 == 0:
        print(f'step: {step}, loss: {loss.item():.2E}')

plt.figure()
plt.plot(losses)
plt.xlabel('Step')
plt.ylabel('Loss')
plt.tight_layout()



Put your learned model to the test: generate and plot some test samples.

In [ ]:
# generate samples from the learned distribution
with torch.no_grad():
    samples = flow_dist.sample(torch.Size([5000, ]))
    samples = unnormalise(samples)

print(samples.max())
print(samples.min())

plt.close('all')
plt.figure(figsize=(10, 10))
show_grayscale_img(img)
plt.scatter(samples[:, 0], samples[:, 1], color='firebrick', label='Generated Samples', alpha=0.5)
# plt.xlim([0, width])
# plt.ylim([0, height])
plt.legend()

We also want to have a look at the log-likelihood of our model!

In [ ]:
# compute the log-likelihood at a discrete grid
num_points = 100
gridpoints = torch.linspace(-1. + 1e-3, 1. - 1e-3, num_points)
X, Y = torch.meshgrid(gridpoints, gridpoints)

# flatten the grid points and evaluate their log-liklihoods
grid = torch.stack((X, Y), dim=-1).flatten(-1)
with torch.no_grad():
    log_likelihood = flow_dist.log_prob(grid)

# finally, let's plot the log-likelihood
zmin = log_likelihood.min().item()
zmax = log_likelihood.max().item()
levels = torch.linspace(zmin, zmax, 200).numpy()
fig, ax = plt.subplots()
surf = ax.contourf(X, Y, log_likelihood, cmap=cm.coolwarm, levels=levels, vmin=zmin, vmax=zmax)
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title('Log-Likelihood')

